In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import importlib
import datasets as d
importlib.reload(d)

import utils 

In [ ]:
print("Cuda OK?:", torch.cuda.is_available())
print("Tensor test:", torch.tensor([3, 4, 5], device='cuda'))

for i in range(torch.cuda.device_count()):
  print(torch.cuda.get_device_name(i))
  print(torch.cuda.get_device_capability(i))
  # print(torch.cuda.get_device_properties(i))

In [ ]:
rod = d.SynRODDataset('data', image_size=118, train=True, download=True)
loader = DataLoader(rod, 64, shuffle=False)

rgb_image, d_image, label = next(iter(loader))

test_image = rgb_image[0]
utils.show_image(test_image)

In [ ]:
f, axarr = plt.subplots(1, 4, figsize=(10, 3))

for r, ax in enumerate(axarr):
  ax.set_title(f"{r*90} deg")
  utils.show_image(utils.rotate_image(test_image, r), ax=ax)